In [2]:
import math
import numpy as np
import matplotlib  as plt
import random

In [4]:
class Value:
    def __init__(self,data,_child=(),_op='',):
        self.data=data
        self.prev=set(_child)
        self._back=lambda:None
        self.grad=0.0
        self._op=_op

    def __repr__(self) :
        return f"Value(data={self.data}"  
      
    def __add__(self,other):
        other =other if isinstance(other,Value) else Value(other)
        out=Value(self.data + other.data,(self,other),'+')

        def _back():
            self.grad+=1.0*out.grad
            other.grad+=1.0*out.grad
            out._back=_back 

        return out
    
    def __mul__(self,other):
        other =other if isinstance(other,Value) else Value(other)
        out=Value(self.data * other.data,(self,other),'*')

        def _back():
            self.grad+=other.grad*out.grad
            other.grad+=self.grad*out.grad
            out._back=_back
        return out
    
    def __pow__(self,other):
        assert isinstance(other,(int,float))
        out=Value(self.data**other,(self,))

        def _back():
            self.grad+=other*(self.data **(other-1))*out.grad
        out._back=_back
        return out
    
    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self,other):
        return self*other**-1
    
    def tanh(self):
        x=self.data
        t=((math.exp(2*x)-1)/math.exp(2*x)+1)
        out=Value(t,(self,),'tanh')

        def _back():
            self.grad=(1-t**2)*out.grad
        out._back=_back    
        return out
    def exp(self):
        x=self.data
        out = Value(math.exp(x),(self,))
        def _back():
            self.grad += out.data*out.grad
        out._back=_back    

    


    def back(self):
    
        topo=[]
        visited =set()
        def build_topo(v):
            visited.add(v)
            for child in v.prev:
                build_topo(child)
            topo.append(v)
        build_topo(self)
        topo        
        o.grad=1.0

        for node in reversed (topo):
            node._back()


In [13]:
x1=Value(2.0)
x2=Value(0.0)
w1=Value(-3.0)
w2=Value(1.0)
b=Value(6)
x1w1=x1*w1
x2w2=x2*w2
x1w1x2w2=x1w1*x2w2
n=x1w1x2w2
o=n.tanh();

In [14]:
import torch

In [16]:
x1= torch.Tensor ([2.0]).double(); x1.requires_grad = True
x2= torch. Tensor ([0.0]).double(); x2.requires_grad = True 
w1= torch. Tensor ([-3.0]).double(); w1.requires_grad = True
w2= torch. Tensor ([1.0]).double();w2.requires_grad = True
b=torch. Tensor ([6.8813735870195432]).double(); b.requires_grad = True
n=x1*w1+x2*w2 + b
o= torch.tanh(n)

print(o.data.item())
o.backward()
print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('wl', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
wl 1.0000002567688737


In [16]:
class Neuron: 
  def __init__(self,nin):
    self.w=[Value(random.uniform(-1,1))for _ in range(nin)]
    self.b=Value(random.uniform(-1,1))

  def __call__(self,x):
    act=sum((wi*xi for wi,xi in zip(self.w,x)),self.b)
    out =act.tanh()
    return out
  
  def preameters(self):
    return [p for neuron in self.neurons for p in neurons.parameters()]

class Layer:

    def __init__(self,nin,nout):
      self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self,x):
      out = [neuron(x) for neuron in self.neurons]
      return out[0]if len(out)== 1 else out
    
    def preameters(self):
      return [p for Layer in self.Layers for p in Layers.parameters()]

class MLP:
  def __init__(self,nin,nouts):
    sz=[nin]+nouts
    self.layers=[Layer(sz[i],sz[i+1]) for i in range(len(nouts))]

  def __call__(self,x):
    for layer in self.layers:
      x=layer(x) 
      return x    
    
x=[2.0,3.0,-1.0]
n=MLP(3,[4,4,1])   
n(x)

[Value(data=1.995754516679734,
 Value(data=-90.00056722745907,
 Value(data=1.937181197192528,
 Value(data=-55.9651935536683]

In [18]:
xs=[
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
]
ys=[1.0,-1.0,-1.0,1.0]
ypred=[n(x) for x in xs]
ypred

[[Value(data=1.995754516679734,
  Value(data=-90.00056722745907,
  Value(data=1.937181197192528,
  Value(data=-55.9651935536683],
 [Value(data=1.981632929064697,
  Value(data=1.6283503510495654,
  Value(data=-53.824617749773104,
  Value(data=1.9858578256018466],
 [Value(data=1.0438475271094503,
  Value(data=-5.766161638932912,
  Value(data=-0.9073130058952537,
  Value(data=1.0478659119549576],
 [Value(data=1.9894954235402413,
  Value(data=0.44765099192328783,
  Value(data=1.1285718774294238,
  Value(data=-12.59671462948738]]

In [19]:
#loss
[(yout-ygt)**2 for ygt,yout in zip(ys,ypred)]

TypeError: unsupported operand type(s) for -: 'list' and 'float'